In [5]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
import re 
import os 
from pathlib import Path
import pandas as pd
import time
import random 
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [2]:

def get_followers_id(soup,url_str='https://weibo.cn'):
    user_names = [a.get_text() for a in soup.find_all('a', href=True) if url_str in a['href'] and a.get_text()]
    user_uids = [a['href'].split('/')[-1] for a in soup.find_all('a', href=True) if url_str in a['href'] and a.get_text()]
    user_uids = [item for item in user_uids if not '=' in item or '.' in item]
    return user_names, user_uids

def save_array2txt(my_array,filename):
    # Open the file for writing
    with open(filename, 'w') as file:
        for item in my_array:
        # Write the string representation of the array to the file
            file.write(item)
            file.write('\n')
    return True

def random_wait(min_seconds=3,max_seconds=10):
    random_seconds = random.randint(min_seconds, max_seconds)
    time.sleep(random_seconds)
    print(f'get next page after {random_seconds} seconds')
    return True

In [ ]:


with open('cookies.txt', 'r', encoding='utf-8') as f:
    cookie = f.read().strip()

# 设置请求头部信息
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Cookie': cookie
}

# 定义需要抓取关注列表的微博用户ID和页数
user_id = '6235235201'
page = 1
user_names = []
user_uids = []


profile_url = f'https://weibo.cn/{user_id}/profile'
profile_response = requests.get(profile_url, headers=headers)
profile_soup = BeautifulSoup(profile_response.content, 'html.parser')
user_name = profile_soup.find('title').text.split(' ')[0][:-3]
print(user_name)



# 构造请求URL
#url = f'https://weibo.cn/u/{user_id}?page={page}'
url = f'https://weibo.cn/{user_id}/follow?page={page}'
# 发送请求
response = requests.get(url, headers=headers)
random_wait(3,4)

# 解析HTML页面并提取关注列表
soup = BeautifulSoup(response.content, 'html.parser') #html5lib  
followings = get_followers_id(soup)
user_names += followings[0]
user_uids += followings[1]

random_wait()

page_next = soup.find('a', string='下页')
while page_next:
    # 获取下一页的URL
    page += 1
    url = f'https://weibo.cn/{user_id}/follow?page={page}'
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    followings = get_followers_id(soup)
    user_names += followings[0]
    user_uids += followings[1]
    page_next = soup.find('a', string='下页')
    random_wait()

user_names = list(set(user_names))
user_uids = list(set(user_uids))

save_array2txt(user_names,f'./following_lists/{user_id}_{user_name}_followers_names.txt')
save_array2txt(user_uids,f'./following_lists/{user_id}_{user_name}_followers_uids.txt')

In [11]:


def download_image(url, filename):
    """
    下载图片
    """
    filename = Path(filename)
    filename.parent.mkdir(parents=True, exist_ok=True)

    r = requests.get(url)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

def download_video(url, path):
    """
    下载视频
    """
    if not os.path.exists(path):
        os.makedirs(path)
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        with open(os.path.join(path, os.path.basename(url)), 'wb') as f:
            for chunk in r.iter_content(1024*1024):
                f.write(chunk)


def get_page_content(user_id,page,headers,curr_time,time_limit,user_folder,original_pic_download=False):
    today_str = curr_time.strftime('%m月%d日')
    next_page = True
    # 构造请求URL
    url = f'https://weibo.cn/{user_id}?filter=1&page={page}'

    # 发送请求
    response = requests.get(url, headers=headers)

    # 解析HTML页面并提取原创微博
    soup = BeautifulSoup(response.content, 'html.parser')
    weibos = soup.find_all('div', class_='c', id=True)

    weibo_content = []

    # 输出原创微博
    for weibo in weibos:
        # 提取微博发布时间
        create_time_str = weibo.find('span', class_='ct').get_text()
        print(f'create_at: {create_time_str}')
        if '分钟前' in create_time_str:
            date_str = today_str
            minutes = create_time_str.split('分钟前')[0]
            if len(minutes)==1:
                time_str = f'{curr_time.hour}:0{minutes}'
            else:
                time_str = f'{curr_time.hour}:{minutes}'
        else:
            date_str,time_str = create_time_str.split(' ')[0],create_time_str.split(' ')[1]
            time_str = time_str[:5]
        print('after process: ',date_str,time_str)
        if date_str=='今天':
            date_str = today_str

        time_str = f'{date_str} {time_str}'
        time = datetime.strptime(time_str, '%m月%d日 %H:%M')
        now = datetime.now()
        time = time.replace(year=now.year)
        if time > time_limit:
            # 提取微博正文
            content = weibo.find('span', class_='ctt').get_text()
            weibo_content.append({'create_at':time,'content':content})
            print(content)
            if original_pic_download:
                imgs = weibo.find_all('img', alt='图片')
                for i,img in enumerate(imgs):
                    img_url = img.get('src')
                    image_folder = Path(os.path.join(user_folder,'img'))
                    image_folder.mkdir(parents=True,exist_ok=True)
                    file_name = os.path.join(image_folder,f'{user_id}_{time.strftime("%Y%m%d%H%M%S")}_{i}.jpg')
                    download_image(img_url, file_name)

        else:
            print('time_str: ',time_str)
            next_page = False
            break
        page_next = soup.find('a', string='下页')
        if not len(page_next):
            next_page = False
    return next_page,weibo_content
    


In [14]:
# 设置请求头部信息和Cookie
with open('cookies.txt', 'r', encoding='utf-8') as f:
    cookie = f.read().strip()

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Cookie': cookie
}

# 定义需要抓取的微博用户ID和页数
user_id = '6028429521'
original_pic_download=True
page = 1
hours = 2
next_page = True

profile_url = f'https://weibo.cn/{user_id}/profile'
profile_response = requests.get(profile_url, headers=headers)
profile_soup = BeautifulSoup(profile_response.content, 'html.parser')
user_name = profile_soup.find('title').text.split(' ')[0][:-3]
print(user_name)
random_wait(3,4)

user_folder = Path(os.path.join(os.getcwd(),'weibo',user_name))
user_folder.mkdir(parents=True,exist_ok=True)

# 计算最近n小时的时间
curr_time = datetime.now()
time_limit = curr_time - timedelta(hours=hours)
print('get weibos until',time_limit.strftime('%m月%d日 %H:%M'))
today_str = time_limit.strftime('%m月%d日')
all_weibo_content = []
next_page,weibo_content = get_page_content(user_id,page,headers,curr_time,time_limit,user_folder)
all_weibo_content.extend(weibo_content)
random_wait(5,20)

while(next_page):
    # 获取下一页的URL
    page += 1
    print(f'get content from page {page}')
    next_page,weibo_content = get_page_content(user_id,page,headers,curr_time,time_limit,user_folder)
    if next_page==False:
        break
    all_weibo_content.extend(weibo_content)
    print(f'find next page {next_page}')
    random_wait(5,20)

    

茶美丽的追剧日常
get next page after 4 seconds
get weibos until 03月17日 15:44
create_at: 48分钟前 来自沦落民间的iPhone 13(粉色)
after process:  03月17日 17:48
裤吃不是吃了绝情丹但是媚成毅的❤️已经藏不住了zb就是如此：谁赚钱媚谁[滑稽] 
create_at: 51分钟前 来自沦落民间的iPhone 13(粉色)
after process:  03月17日 17:51
#姜广涛涉嫌刑事犯罪#姜广涛如果被列为劣迹人员，他所配音的作品都下架，会有多少人受连累 
create_at: 今天 16:28 来自沦落民间的iPhone 13(粉色)
after process:  今天 16:28
#2022微博十大热门电视剧#刘亦菲，陈晓领衔的梦华录！基本上算去年的剧王了吧？各个榜单都是梦华录第一。 
create_at: 今天 16:27 来自沦落民间的iPhone 13(粉色)
after process:  今天 16:27
#微博之夜第三波官宣阵容#刘宇真滴蛮帅的，他感觉do something了，但又不知道do了哪里，整个人都焕然一新似的，像漫画里走出来的人，你们能get到嘛？ 
create_at: 今天 16:25 来自沦落民间的iPhone 13(粉色)
after process:  今天 16:25
#微博白皮书影视作品热门cp#热门CP：顾盼生辉、苍兰夫妇、疑商夫妇、沉香夫妇、命韵峋环。为什么没有余生夫妇？？？？？？？？？ 
create_at: 今天 16:14 来自沦落民间的iPhone 13(粉色)
after process:  今天 16:14
#杨紫粉丝娱乐白皮书寄语##微博之夜# 杨紫确定出席2022微博之夜 
create_at: 今天 16:09 来自沦落民间的iPhone 13(粉色)
after process:  今天 16:09
#微博之夜舞台名场面##微博之夜# #群星为娱乐白皮书打call# #微博之夜舞台名场面# 六大舞台名场面：哪个让你们最印象深刻？ 
create_at: 今天 15:51 来自沦落民间的iPhone 13(粉色)
after process:  今天 15:51
#迪丽热巴驼色大衣造型#

In [23]:
content_df = pd.DataFrame(all_weibo_content)
content_df['user_uid'] = user_id
content_df['user_name'] = user_name
content_df['last_check'] = time_limit.strftime('%Y-%m-%d %H:%M')
content_df['check_before'] = hours
content_df

,create_at,content,user_uid,user_name,last_check,check_before
0,2023-03-17 17:48:00,裤吃不是吃了绝情丹但是媚成毅的❤️已经藏不住了zb就是如此：谁赚钱媚谁[滑稽],6028429521,茶美丽的追剧日常,2023-03-17 15:44,2
1,2023-03-17 17:51:00,#姜广涛涉嫌刑事犯罪#姜广涛如果被列为劣迹人员，他所配音的作品都下架，会有多少人受连累,6028429521,茶美丽的追剧日常,2023-03-17 15:44,2
2,2023-03-17 16:28:00,#2022微博十大热门电视剧#刘亦菲，陈晓领衔的梦华录！基本上算去年的剧王了吧？各个榜单都是...,6028429521,茶美丽的追剧日常,2023-03-17 15:44,2
3,2023-03-17 16:27:00,#微博之夜第三波官宣阵容#刘宇真滴蛮帅的，他感觉do something了，但又不知道do了...,6028429521,茶美丽的追剧日常,2023-03-17 15:44,2
4,2023-03-17 16:25:00,#微博白皮书影视作品热门cp#热门CP：顾盼生辉、苍兰夫妇、疑商夫妇、沉香夫妇、命韵峋环。为...,6028429521,茶美丽的追剧日常,2023-03-17 15:44,2
5,2023-03-17 16:14:00,#杨紫粉丝娱乐白皮书寄语##微博之夜# 杨紫确定出席2022微博之夜,6028429521,茶美丽的追剧日常,2023-03-17 15:44,2
6,2023-03-17 16:09:00,#微博之夜舞台名场面##微博之夜# #群星为娱乐白皮书打call# #微博之夜舞台名场面# ...,6028429521,茶美丽的追剧日常,2023-03-17 15:44,2
7,2023-03-17 15:51:00,#迪丽热巴驼色大衣造型##与君初相识播出一周年# 迪丽热巴走御姐风真的yyds，黑长直还蛮有...,6028429521,茶美丽的追剧日常,2023-03-17 15:44,2


In [24]:

def contains_chinese_keywords(text, keywords):
    query_result = []
    for keyword in keywords:
        # 使用正则表达式查找中文关键词
        pattern = re.compile(keyword)
        match = pattern.search(text)
        if match:
            query_result.append(keyword)
    return query_result



In [17]:
keywords = ["成毅"]
for index,row in content_df.iterrows():
    query_keywords_result = contains_chinese_keywords(row['content'],keywords)
    if len(query_keywords_result):
        print(row['create_at'],row['content'])

2023-03-17 17:48:00 裤吃不是吃了绝情丹但是媚成毅的❤️已经藏不住了zb就是如此：谁赚钱媚谁[滑稽] 
